In [7]:
import asyncio
from typing import Optional
from contextlib import AsyncExitStack, ExitStack

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from anthropic import Anthropic
from dotenv import load_dotenv
        
server_params = StdioServerParameters(
    **{
        "command": "docker",
        "args": [
        "run",
        "--rm",
        "-i",
        "--mount", "type=bind,src=/Users/sabrieyuboglu/code/cartridges,dst=/workspace",
        "mcp/git"
        ]
    }
)

stdio, write = stdio_client(server_params)
session = ClientSession(stdio, write)
    

TypeError: cannot unpack non-iterable _AsyncGeneratorContextManager object

In [6]:
client = MCPClient()

asyncio.run(client.connect_to_server("mcp_server_git"))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
"""
Script to fetch every message from the Slack channel defined by the
`SLACK_CHANNEL_ID` environment variable.

Requirements:
    pip install slack_sdk python-dotenv
    # and create a .env file containing at least
    # SLACK_BOT_TOKEN=xoxb-***
    # SLACK_CHANNEL_ID=C********
"""

from __future__ import annotations

import os
from typing import Any, Dict, List

from dotenv import load_dotenv
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError


SLACK_BOT_TOKEN = os.environ["SLACK_BOT_TOKEN"]
SLACK_TEAM_ID = os.environ["SLACK_TEAM_ID"]

if not SLACK_BOT_TOKEN or not SLACK_CHANNEL_ID:
    raise EnvironmentError(
        "Both SLACK_BOT_TOKEN and SLACK_CHANNEL_ID must be set "
        "as environment variables (or in a .env file)."
    )

client = WebClient(token=SLACK_BOT_TOKEN)


def fetch_all_messages(channel_id: str) -> List[Dict[str, Any]]:
    """
    Retrieve every message in the specified Slack channel.

    Slack paginates results, returning up to 200 messages per call.
    This function continues requesting pages until no more messages remain.

    Returns
    -------
    List[Dict[str, Any]]
        A list of raw Slack message objects.
    """
    messages: List[Dict[str, Any]] = []
    cursor: str | None = None

    while True:
        try:
            resp = client.conversations_history(
                channel=channel_id,
                limit=200,
                cursor=cursor,
            )
        except SlackApiError as exc:
            # Abort on any Slack API error and surface the message
            raise RuntimeError(
                f"Failed to fetch messages: {exc.response['error']}"
            ) from exc

        messages.extend(resp.get("messages", []))

        cursor = resp.get("response_metadata", {}).get("next_cursor")
        if not cursor:
            break  # No further pages

    return messages


all_msgs = fetch_all_messages(SLACK_CHANNEL_ID)
print(f"Fetched {len(all_msgs)} messages from channel {SLACK_CHANNEL_ID}\n")

# Print the text of the last 5 messages for quick inspection
for msg in all_msgs[-2:]:
    ts = msg.get("ts")
    user = msg.get("user")
    text = msg.get("text")
    print(f"[{ts}] {user}: {text}")


Fetched 67 messages from channel C165G5W4D

[1742314253.754089] U1EECV1QR: Who is handling the upgraded bathroom art? :pray:
[1742230869.057299] USLACKBOT: This message was deleted.
